# Sesión 11 — CNN Fundamentals 

En esta sesión desarrollaremos intuición práctica de **convoluciones**:
- qué hace un filtro
- cómo aparecen los **feature maps**
- qué efectos tienen **stride** y **padding**
- cómo se conectan con una CNN real

Pregunta guía: **¿Por qué un filtro aprendido puede reutilizarse en toda la imagen?**


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


## 1) Convolución “a mano” en 2D (intuición)

Vamos a crear una imagen sintética con un borde, y aplicarle filtros clásicos:
- borde vertical
- borde horizontal

Nota: aquí usamos filtros fijos para entender el mecanismo.


In [ ]:
# Imagen sintética: un "borde" vertical
img = torch.zeros(1, 1, 32, 32)  # (N,C,H,W)
img[:, :, :, :16] = 0.0
img[:, :, :, 16:] = 1.0

plt.figure(figsize=(3,3))
plt.imshow(img[0,0], cmap="gray")
plt.title("Imagen sintética (borde vertical)")
plt.axis("off")
plt.show()


In [ ]:
# Definir kernels (filtros) 3x3 estilo Sobel (aprox)
k_vert = torch.tensor([[
    [-1., 0., 1.],
    [-2., 0., 2.],
    [-1., 0., 1.],
]], dtype=torch.float32)  # (1,3,3)

k_horz = torch.tensor([[
    [-1., -2., -1.],
    [ 0.,  0.,  0.],
    [ 1.,  2.,  1.],
]], dtype=torch.float32)

# Conv2d espera pesos con forma (out_channels, in_channels, kH, kW)
W = torch.stack([k_vert, k_horz], dim=0).unsqueeze(1)  # (2,1,3,3)

W.shape


In [ ]:
# Aplicar conv2d con padding para conservar tamaño
out = F.conv2d(img, W, bias=None, stride=1, padding=1)  # (1,2,32,32)

fig, axes = plt.subplots(1, 3, figsize=(10, 3))
axes[0].imshow(img[0,0], cmap="gray"); axes[0].set_title("Entrada"); axes[0].axis("off")
axes[1].imshow(out[0,0].detach(), cmap="gray"); axes[1].set_title("Filtro vertical"); axes[1].axis("off")
axes[2].imshow(out[0,1].detach(), cmap="gray"); axes[2].set_title("Filtro horizontal"); axes[2].axis("off")
plt.show()

# Pregunta: ¿por qué el filtro vertical responde fuerte cerca del borde vertical?


## 2) Efecto de stride y padding

- `stride` reduce resolución (submuestreo)
- `padding` evita que la imagen “se encoja” demasiado y ayuda con bordes


In [ ]:
# Comparar tamaños de salida
out_s1_p0 = F.conv2d(img, W[:1], stride=1, padding=0)  # solo 1 filtro
out_s1_p1 = F.conv2d(img, W[:1], stride=1, padding=1)
out_s2_p1 = F.conv2d(img, W[:1], stride=2, padding=1)

print("stride=1, padding=0:", tuple(out_s1_p0.shape))
print("stride=1, padding=1:", tuple(out_s1_p1.shape))
print("stride=2, padding=1:", tuple(out_s2_p1.shape))

# Pregunta: ¿qué ganas y qué pierdes al pasar de stride=1 a stride=2?


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
axes[0].imshow(out_s1_p0[0,0], cmap="gray"); axes[0].set_title("s=1, p=0"); axes[0].axis("off")
axes[1].imshow(out_s1_p1[0,0], cmap="gray"); axes[1].set_title("s=1, p=1"); axes[1].axis("off")
axes[2].imshow(out_s2_p1[0,0], cmap="gray"); axes[2].set_title("s=2, p=1"); axes[2].axis("off")
plt.show()


## 3) Una capa Conv2d real aprende los filtros

Creamos una capa `nn.Conv2d` y observamos:
- su forma de pesos
- cómo produce múltiples mapas

Todavía no entrenamos; esto es para ver estructura.


In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
conv.weight.shape, conv.bias.shape


In [ ]:
# Aplicar a una imagen MNIST para ver feature maps
try:
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader
    tfm = transforms.Compose([transforms.ToTensor()])
    ds = datasets.MNIST(root="./data", train=True, download=True, transform=tfm)
    loader = DataLoader(ds, batch_size=1, shuffle=True)
    x, y = next(iter(loader))
except Exception as e:
    raise RuntimeError(f"No pude cargar MNIST (torchvision). Error: {e}")

with torch.no_grad():
    feats = conv(x)  # (1, 8, 28, 28)

print("entrada:", tuple(x.shape), "feature maps:", tuple(feats.shape), "label:", int(y.item()))

# Mostrar algunos mapas
fig, axes = plt.subplots(2, 4, figsize=(10, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(feats[0, i], cmap="gray")
    ax.set_title(f"map {i}")
    ax.axis("off")
plt.suptitle("Feature maps (Conv2d sin entrenar)")
plt.show()

# Pregunta: sin entrenar, ¿por qué algunos mapas ya muestran contrastes?


## 4) Mini-CNN (bloques)

Patrón típico:
- Conv → ReLU → Pool
- repetir
- Flatten → Linear

En Session 12 construiremos una CNN completa para clasificación.
Aquí solo verificamos shapes.


In [ ]:
mini = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),  # 28x28 -> 14x14
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),  # 14x14 -> 7x7
)

with torch.no_grad():
    h = mini(x)  # x es (1,1,28,28)
h.shape


In [ ]:
# ¿Cuántas características quedan después de los bloques conv?
flatten_dim = h.numel()
flatten_dim


## Cierre

Hoy construimos intuición:

- **convolución** = filtro que detecta patrones locales
- **pesos compartidos** = el mismo patrón puede aparecer en cualquier lugar
- **stride/padding/pooling** controlan resolución y robustez
- apilar convs crea una **jerarquía de representaciones**

Pregunta final: **¿Qué supuesto sobre imágenes hace que una CNN generalice mejor que un MLP?**
